In [1]:
import numpy as np
from fractions import *

from time import perf_counter, time

import sys

sys.path.insert(1, "/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy")

import monodromy

from monodromy.coordinates import monodromy_alcove, monodromy_alcove_c2, monodromy_to_positive_canonical_polytope, rho_reflect
from monodromy.elimination import cylinderize, project
from monodromy.polytopes import *
from monodromy.static import qlr_polytope
from monodromy.method import *

from itertools import count

import os

import matplotlib.pyplot as plt

from scipy import stats

from functools import reduce
from itertools import combinations

from sympy import Matrix

/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy/lrslib-071b/lrs


Is the strong polytope just the weak polytope but scaled chopped off and reflected?

In [63]:
# Define grand alcove
def grand_alcove(n): 
    cylinderized_alcove = cylinderize(monodromy_alcove_c2, [0, 1, 2, 3], n+4)
    ineq = []
    for i in range(1, n+1):
        l1, l2 = [0]*(n+4), [0]*(n+4)
        l1[0] = 1
        l1[-i] = -2
        l2[-i] = 1
        ineq.append(l1)
        ineq.append(l2)
    
    xy_poly = Polytope([ConvexPolytope(inequalities=ineq, name = f"grand_polytope_{n}")])
    cylinderized_alcove = monodromy_to_positive_canonical_polytope(cylinderized_alcove.intersect(xy_poly)).reduce()
    
    return cylinderized_alcove.convex_subpolytopes[0]

In [3]:
def model_polytope(angle, n=3, poly = 0):
    """Return a polytope that is a scaled version of of the weak case"""
    
    ineq = sorted(polytope_dict[Fraction(1, 5)][n].convex_subpolytopes[poly].inequalities)
    print(len(ineq))
    for i in range(1, n+1):
        ineq[-i][0] = Fraction(angle).numerator
        ineq[-i][-i] = -Fraction(angle).denominator

    polytope = ConvexPolytope(ineq)
    
    return polytope

First, we find out if the strong model polytope contains convex component 0 of strong polytope or not. We also find that the model polytope is always in the Weyl chamber for (angle <=1/2)

In [9]:
d = {0:3, 1:2, 2:1, 3:0}

In [65]:
angle, n, poly = Fraction(1, 2), 4, 0
strong_polytope1 = polytope_dict[angle][n].convex_subpolytopes[poly]
strong_polytope2 = polytope_dict[angle][n].convex_subpolytopes[d[poly]]

strong_model_polytope = model_polytope(angle, n, poly)

36


In [66]:
# Check if grand_alcove contains the strong_model
grand_alcove(4).contains(strong_model_polytope)

True

In [67]:
# Check strong_model containing strong poly (first)
strong_model_polytope.contains(strong_polytope1)

True

In [68]:
# Check strong_model containing strong poly (second)
strong_model_polytope.contains(strong_polytope2)

True

In [23]:
strong_polytope3 = polytope_dict[angle][n].convex_subpolytopes[5]
# Check strong_model containing strong poly (second)
strong_model_polytope.contains(strong_polytope3)

True

In [29]:
# Check the other way round
strong_polytope3.contains(strong_model_polytope)

False

Now, we find out if the volume of the strong model polytope is the sum of volume of component 0 and 3. Or rather, if we take the volume of all the disjoint components, is it the volume of the strong component?
- The answer seems to be no, but the volume union AcupB is very close to the volume of the strong model polytope

In [30]:
float(strong_model_polytope.volume.volume)

0.0023240412540723593

In [39]:
# |A ∪ B| = |A| + |B| - |A ∩ B| - |A ∩ C| - |B ∩ C| + |A ∩ B ∩ C|
A = float(strong_polytope1.volume.volume)
B = float(strong_polytope2.volume.volume)
C = float(strong_polytope3.volume.volume)
AcapB = float(strong_polytope1.intersect(strong_polytope2).volume.volume)
AcapC = float(strong_polytope1.intersect(strong_polytope3).volume.volume)
BcapC = float(strong_polytope2.intersect(strong_polytope3).volume.volume)
AcapBcapC = float(strong_polytope1.intersect(strong_polytope2).intersect(strong_polytope3).volume.volume)
AcupB = A+B+C-AcapB-AcapC-BcapC+AcapBcapC

In [40]:
AcupB

0.002311827046437145

In [41]:
0.002311827046437145/0.0023240412540723593*100

99.47444101459854